In [235]:
#My Documents
idcard = {
    "name": "John Doe",
    "age": "22",
    "id": "12345678",
    "nif": "258852555"
}

# Tabela de nomes para a empresa ficar com par chave
# pública -> nickname
name_table = {}

verified = False

stuff= ""

import os
import sys
import random
import pickle
from OpenSSL import crypto
from cryptography.hazmat.primitives.asymmetric import dsa
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.exceptions import *

In [222]:
#Funções auxiliares para criar certificados

def createCACert():
    
    ca_key = crypto.PKey()
    ca_key.generate_key(crypto.TYPE_RSA, 2048)
    
    ca_cert = crypto.X509()
    ca_cert.set_version(2)
    ca_cert.set_serial_number(random.randint(50000000,100000000))
    
    ca_subj = ca_cert.get_subject()
    ca_subj.commonName = b'Entidade Certificadora'
    
    ca_cert.add_extensions([
        crypto.X509Extension(b'subjectKeyIdentifier', False, b'hash', subject=ca_cert),
    ])

    ca_cert.add_extensions([
        crypto.X509Extension(b'authorityKeyIdentifier', False, b'keyid:always', issuer=ca_cert),
    ])

    ca_cert.add_extensions([
        crypto.X509Extension(b'basicConstraints', False, b'CA:TRUE'),
        crypto.X509Extension(b'keyUsage', False, b'keyCertSign, cRLSign'),
    ])
    
    ca_cert.set_issuer(ca_subj)
    ca_cert.set_pubkey(ca_key)
    
    ca_cert.gmtime_adj_notBefore(0)
    ca_cert.gmtime_adj_notAfter(10*365*24*60*60)
    
    ca_cert.sign(ca_key,'sha256')
    
    # Save certificate
    with open("ca.crt","wb") as f:
        f.write(crypto.dump_certificate(crypto.FILETYPE_PEM,ca_cert))
        
    with open("ca-private.key",'wb') as f:
        f.write(crypto.dump_privatekey(crypto.FILETYPE_PEM,pkey=ca_key))
    
    with open("ca-public.key",'wb') as f:
        f.write(crypto.dump_publickey(crypto.FILETYPE_PEM,pkey=ca_key))
    
       

def createClientCert(doc):
    
    file= open("ca.crt","rb")
    ca_cert = crypto.load_certificate(crypto.FILETYPE_PEM,file.read())
    
    file= open("ca-private.key","rb")
    ca_key = crypto.load_privatekey(crypto.FILETYPE_PEM, file.read())
    
    
    client_key = crypto.PKey()
    client_key.generate_key(crypto.TYPE_RSA, 2048)
    
    client_cert = crypto.X509()
    client_cert.set_version(2)
    client_cert.set_serial_number(random.randint(50000000,100000000))
    
    client_subj = client_cert.get_subject()
    client_subj.commonName = "John Doe"
    
    client_cert.add_extensions([
        crypto.X509Extension(b'basicConstraints',False,b'CA:FALSE'),
        crypto.X509Extension(b'subjectKeyIdentifier', False, b'hash', subject=client_cert),
    ])
    
    client_cert.add_extensions([
        crypto.X509Extension(b'authorityKeyIdentifier', False, b'keyid:always', issuer=ca_cert),
        crypto.X509Extension(b'extendedKeyUsage', False, b'clientAuth'),
        crypto.X509Extension(b'keyUsage', False, b'digitalSignature'),
    ])
    
    client_cert.set_issuer(ca_cert.get_subject())
    client_cert.set_pubkey(client_key)
    
    
    ca_cert.gmtime_adj_notBefore(0)
    ca_cert.gmtime_adj_notAfter(10*365*24*60*60)
    
    client_cert.sign(ca_key,'sha256')
    
    #doc = pickle.dumps(client_doc)
    
    signature = crypto.sign(ca_key,doc,'sha256')
    
    
    with open("client.crt", "wb") as f:
        f.write(crypto.dump_certificate(crypto.FILETYPE_PEM, client_cert))
        
    with open("client-private.key",'wb') as f:
        f.write(crypto.dump_privatekey(crypto.FILETYPE_PEM,pkey=client_key))
    
    with open("client-public.key",'wb') as f:
        f.write(crypto.dump_publickey(crypto.FILETYPE_PEM,pkey=client_key))
        
    return signature    

#Verificar o certificado    
def verifyCert(signature):
    
    file= open("ca.crt","rb")
    ca_cert = crypto.load_certificate(crypto.FILETYPE_PEM,file.read())
    
    
    try:
        doc = pickle.dumps(idcard)
        
        crypto.verify(ca_cert,signature,doc,'sha256')
        
        global verified
        verified = True
        
        
    except crypto.Error:
        print("Falhou")
         
    
    
#createCACert() 
#createClientCert(idcard)

In [234]:
#1º Connection Client <-> CA

def Client(conn):
    
    #pede o certificado
    signed = askCert(conn)
    global stuff
    stuff = signed
    
    #começa a conecção com a Entidade
    BiConn(Client2,Ent,timeout=30).auto()

    
def Cert(conn):
    
    g = sendCert(conn)
    
    
    
    
#2º Connection Client <-> Entity(Requester)   
def Client2(conn2):
    
    #pede para verificar o certificado à entidade
    global stuff
    c = verifyMe(conn2,stuff)
    
    
def Ent(conn2):
    #verifica o certificado
    verifyClient(conn2)
    

#3º Connection Entity(Requester) <-> CA
def Ent2(conn3):
    #Envia o certificado à CA
    print("Please verify Client")
    global stuff
    conn3.send(stuff)
    
    #recebe verificação
    ver = conn3.recv()
    
    if(ver == False):
        print("Can't exchange documents")
    else:
        file = open("chaves/publicas/pbClient.pem","rb")
        dsa_pb_cert = file.read()
        name_table[dsa_pb_cert] = "John Doe"
        #Inicia Coneção final com o cliente
        BiConn(Ent3,Client3,timeout=30).auto()
        
def Cert2(conn3):
    #Recebe a assinatura que a entidade enviou
    signature = conn3.recv()
    print("Verifying please wait")
    #Verifica o certificado
    verifyCert(signature)
    global verified
    print(verified)
    if(verified == False):
        print("Verification Failed")
        conn3.send(verified)
    else:
        print("Verification success")
        conn3.send(verified)
        
#4º Connection Entity(Requester) <-> Client      
def Ent3(conn4):
    #Pede informação
    print("Please send age")
    info = "age"
    conn4.send(info)
    
    #Recebe informação assinada
    sign = conn4.recv()
    age = conn4.recv()
    pb = conn4.recv()
    
    print(sign)
    print(age)
    name = name_table[pb]
    print(name)
    file = open("chaves/publicas/pbClient.pem","rb")
    dsa_pb_client = deserialize(file.read())
    
    try:
        dsa_pb_client.verify(sign, age, hashes.SHA256())
        print("Thank you " + name + " for confirming your age " + pickle.loads(age))
    except InvalidSignature:
        print("Assinatura DSA falhou")
    #pub_key = conn4.recv()
    
    
def Client3(conn4):
    #Recebe o pedido de informação
    info_received=conn4.recv()
    print("Signing age info")
    #Assina a informação DSA
    info = idcard[info_received]
    
    snd = pickle.dumps(info)
    
    
    file = open("chaves/privadas/pkClient.pem","rb")
    dsa_pk_client = deserializeDSA(file.read(),"1234")
    
    file = open("chaves/publicas/pbClient.pem","rb")
    pb = file.read()
    
    
    sign = dsa_pk_client.sign(snd,hashes.SHA256())
    
    print(sign)
    
    conn4.send(sign)
    conn4.send(snd)
    conn4.send(pb)
    
    
    

In [233]:
BiConn(Client,Cert,timeout=30).auto()

Waiting for request
Asking CA for certification
Client Certificate created and document Signed
Please verify Client
Verifying please wait
True
Verification success
Hello
Please send age
Signing age info
b'0-\x02\x15\x00\x87\xac\x81\xceG\xba\xccf\xe9\x994\xb1M\x12\tm\xf4yH\xc2\x02\x14\x04\xa3\xc8\xbezS\x00\x06\xfdrj\xaf1\xbc\x1f\xab\x160~r'
b'0-\x02\x15\x00\x87\xac\x81\xceG\xba\xccf\xe9\x994\xb1M\x12\tm\xf4yH\xc2\x02\x14\x04\xa3\xc8\xbezS\x00\x06\xfdrj\xaf1\xbc\x1f\xab\x160~r'
b'\x80\x03X\x02\x00\x00\x0022q\x00.'
John Doe
Thank youJohn Doefor confirming your age22


In [225]:
#Funções auxiliares de verificação de certificado
def verifyMe(conn2,stuff):
    
    conn2.send(stuff)
    
    
def verifyClient(conn2):
    
    signature = conn2.recv()
    
    BiConn(Ent2,Cert2,timeout=30).auto()


In [226]:
#Cliente pede o certificado
def askCert(conn):
    
    print("Asking CA for certification")
    
    doc = pickle.dumps(idcard)
    
    conn.send(doc)
    
    signature = conn.recv()
    
    return signature
    
    
#CA envia certificado
def sendCert(conn):
    
    print("Waiting for request")
    
    doc = conn.recv()
    
    signature = createClientCert(doc)
    
    print("Client Certificate created and document Signed")
    
    conn.send(signature)
    

In [227]:
#Funções de serialização e desserialização de chaves
def serialize(key):
    return key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo)

def deserialize(key):
    return serialization.load_pem_public_key(
        key,
        backend=default_backend())

def serializeDSA(key,password):
    return key.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.PKCS8,
        encryption_algorithm=serialization.NoEncryption()
    )

def deserializeDSA(key,password):
    return serialization.load_pem_private_key(
        key,
        password=None,
        backend=default_backend()
    )

In [68]:
from multiprocessing import Pipe, Process

class BiConn(object):
    def __init__(self,left,right,timeout=None):
        """
        left : a função que vei ligar ao lado esquerdo do Pipe
        right: a função que vai ligar ao outro lado
        timeout: (opcional) numero de segundos que aguarda pela terminação do processo
        """
        left_end, right_end = Pipe()
        self.timeout=timeout
        self.lproc = Process(target=left, args=(left_end,))       # os processos ligados ao Pipe
        self.rproc = Process(target=right, args=(right_end,))
        self.left  = lambda : left(left_end)                       # as funções ligadas já ao Pipe
        self.right = lambda : right(right_end)
    
    def auto(self, proc=None):
        if proc == None:             # corre os dois processos independentes
            self.lproc.start()
            self.rproc.start()  
            self.lproc.join(self.timeout)
            self.rproc.join(self.timeout)
        else:                        # corre só o processo passado como parâmetro
            proc.start(); proc.join()
    
    def manual(self):   #  corre as duas funções no contexto de um mesmo processo Python
        self.left()
        self.right()

In [39]:
#Generate Keys for Identity Authority exchange

password = "1234"

pkClient = dsa.generate_private_key(
    key_size=1024,
    backend=default_backend()
)

pkCert = dsa.generate_private_key(
    key_size=1024,
    backend=default_backend()
)

pbClient = pkClient.public_key()
pbCert = pkCert.public_key()

serialized_pk_client = serializeDSA(pkClient,"1234")
serialized_pk_cert = serializeDSA(pkCert,"1234")

serialized_pb_client = serialize(pbClient)
serialized_pb_cert = serialize(pbCert)


#print(serialized_pb_cert)

#Save to file

file1 = open("chaves/privadas/pkClient.pem","wb")
file1.write(serialized_pk_client)

file2 = open("chaves/privadas/pkCert.pem","wb")
file2.write(serialized_pk_cert)

file3 = open("chaves/publicas/pbCert.pem","wb")
file3.write(serialized_pb_cert)

file4 = open("chaves/publicas/pbClient.pem","wb")
file4.write(serialized_pb_client)




654